## LangChain
https://python.langchain.com/en/latest/modules/memory/examples/adding_memory_chain_multiple_inputs.html

LangChain 是一种LLMs接口框架，它允许用户围绕大型语言模型快速构建应用程序和管道。 
它直接与 OpenAI 的 GPT-3 和 GPT-3.5 模型以及 Hugging Face 的开源替代品（如 Google 的 flan-t5 模型）集成。

LangChain的核心思想是我们可以将不同的组件“链接”在一起，以围绕 LLM 创建更高级的用例。 链(chain)可能由来自多个模块的多个组件组成。

要实现的功能，是上传参考文档（txt格式），ChatGPT 就这些文档展开功能。如果问其中的信息，抽取文章结构，续写，改写，模仿改写等等。
或者是，给定义文档，让ChatGPT按定义活动。
可用于聊天机器人、生成式问答(GQA)、本文摘要等。

这里我们会用到一种称为文本嵌入(Embeddings)的技术：

准备三个文案，放在"./data/{file}"

In [ ]:
#我们需要安装如下依赖包：
#如果安装了conda ，先激活环境，然后改成用conda安装
#! conda install lxml
#! conda install -c conda-forge hnswlib
#! conda install langchain
#! conda install openai
#! conda install chromadb
#! conda install jieba
#! conda install unstructured 
#! pip install nltk
! conda activate lanChain
! conda env list

In [ ]:
#下载自然语言处理工具包
#打开cmd  pip install nltk
#打开cmd黑色窗口

#输入命令：python -m nltk.downloader -u http://nltk.github.com/nltk_data/
#会跳出一个窗口：点击Download 下载会进行一段时间 当看到蓝色标记表明下载成功

# 输入python 再输入import nltk 再输入nltk.download()  出来解决方案1的窗口再点击点击Download 
import nltk  
nltk.download()
from nltk.book import *

#如果出现text1 - text9 就成功了。
#在终端和anaconda prompt中都已经成功。就这个 Jupyte不成功。


In [2]:
# 导入依赖包
import os
from langchain.embeddings.openai import OpenAIEmbeddings  #文本Embedding
from langchain.vectorstores import Chroma   #向量存储
from langchain.text_splitter import TokenTextSplitter #文本切分
from langchain.llms import OpenAI
from langchain.chains import ChatVectorDBChain #聊天向量数据库链
from langchain.document_loaders import DirectoryLoader #文件加载器
import jieba as jb  #汉字切词 文字向量预处理


# 文档预处理
由于中文的语法的特殊性，对于中文的文档必须要做一些预处理工作：
词语的拆分，也就是要把中文的语句拆分成一个个基本的词语单位，
这里我们会用的一个分词工具：jieba，它会帮助我们对资料库中的所有文本文件进行分词处理。
不过我们首先将这3个时事新闻的文本文件放置到Data文件夹下面，然后在data文件夹下面再建一个子文件夹:cut, 用来存放被分词过的文档：
注意编码，必须是utf-8

In [4]:
files=['yl.txt','xm.txt','ch.txt','xj.txt']

for file in files:
    #读取data文件夹中的中文文档
    my_file=f"./data/{file}"
    with open(my_file,"r",encoding='utf-8') as f:  
        data = f.read()
    
    #对中文文档进行分词处理
    cut_data = " ".join([w for w in list(jb.cut(data))])
    #分词处理后的文档保存到data文件夹中的cut子文件夹中
    cut_file=f"./data/cut/cut_{file}"
    with open(cut_file, 'w',encoding='utf-8') as f:   
        f.write(cut_data)
        f.close()

# 文本嵌入(Embeddings)
接下来我们要按照LangChain的流程来处理这些经过中文分词处理的数据，首先是加载文档,然后要对文档进行切块处理，
切块处理完成以后需要调用openai的Embeddings方法进行文本嵌入操作和向量化操作，
最后我们需要创建一个聊天机器人的chain, 这个chain可以加载openai的各种语言模型，这里我们加载gpt-3.5-turbo模型。

会需要很多包要安装。最难的是这个 import hnswlib ,不能通过pip 安装。
WIndows下使用anaconda安装hnswlib包的话，使用如下语句即可：conda install -c conda-forge hnswlib
如果是在linux下：
    先 git clone https://github.com/nmslib/hnswlib.git
    用安装安装pybind11，直接pip install pybind11即可
    切换到hnswlib文件下，python setup.py install

In [ ]:
#加载文档
loader = DirectoryLoader('./data/cut',glob='**/*.txt')
docs = loader.load()

#文档切块
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
doc_texts = text_splitter.split_documents(docs)

#调用openai Embeddings
#这里填写你的key，或者在环境中写
#set OPENAI_API_KEY = "your key"
#os.environ["OPENAI_API_KEY"] = "sk-写在.env中"
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])

#向量化 这个动作要每次做吗？还是第一次做完，文件不变就不用做第二次，直接加载？
vectordb = Chroma.from_documents(doc_texts, embeddings, persist_directory="./data/cut")
vectordb.persist()

#创建聊天机器人对象chain
chain = ChatVectorDBChain.from_llm(OpenAI(temperature=0, model_name="gpt-3.5-turbo"), vectordb, return_source_documents=True)

# 创建聊天函数
接下来我们要创建一个聊天函数，用来让机器人回答用户提出的问题，
这里我们让机器人每次只针对当前问题进行回答，并没有将历史聊天记录保存起来一起喂给机器人。

有个memery模块，可以存储history。
这里可以改进，现在重点先看核心功能。

In [ ]:
def get_answer(question):
  chat_history = []
  result = chain({"question": question, "chat_history": chat_history});
  return result["answer"]

question = "2023年3月都有什么新闻"
print(get_answer(question))

# 输入 prompts 模板设置
在上面根据公司生产的产品生成公司名字的应用中，一种让用户输入更简单的方式是仅让客户输入公司生产的产品即可，不需要输入整个语句，这需要对 prompts 设置模板：


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
print(prompt.format(product="colorful socks")) # 返回 What is a good name for a company that makes colorful socks?
text = prompt.format(product="colorful socks")
print(llm(text)) # 返回 Socktastic！
text = prompt.format(product="chocolates")
print(llm(text)) # 返回 ChocoDelightz！

# Memory 功能: 在 LLM 交互中记录交互的历史状态，并基于历史状态修正模型预测
该实现基于论文： MemPrompt

即当模型出错了之后，用户可以反馈模型错误的地方，然后这些反馈会被添加到 memory 中，以后遇到类似问题时模型会提前找到用户的反馈，从而避免犯同样的错
在这里插入图片描述
对话任务中的 ConversationChain 示例（ConversationBufferMemory 模式），verbose=True 会输出对话任务中的 prompt，可以看到之前聊天会作为短期 memory 加在 prompt 中，从而让模型能有短时间的记忆能力：




In [ ]:
from langchain import OpenAI, ConversationChain
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)
conversation.predict(input="Hi there!")

 # 返回如下

#> Entering new ConversationChain chain...
#Prompt after formatting:
#The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific #details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

#Current conversation:

#Human: Hi there!
#AI:

#> Finished chain.

# Out[53]: " Hi there! It's nice to meet you. How can I help you today?"
conversation.predict(input="I'm doing well! Just having a conversation with an AI.") 

# 返回如下
#Prompt after formatting:
#The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific #details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

#Current conversation:

#Human: Hi there!
#AI:  Hi there! It's nice to meet you. How can I help you today?
#Human: I'm doing well! Just having a conversation with an AI.
#AI:

#> Finished chain.
#Out[54]: " That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"


LangChain 这里看起来没有直接在 QA 任务中集成 memprompt，不过可以基于对话任务来测试之前 QA 任务中出错的问题，可以看到基于 memprompt 确实可以利用用户的反馈来修正模型预测结果：
conversation.predict(input="what word is similar to good?") # 返回 ' Synonyms for "good" include excellent, great, fine, and superb.'
conversation.predict(input="similar to means with similar pronunciation") # 返回 ' Ah, I see. Synonyms for "good" with similar pronunciation include wood, hood, and should.'
这里的实现看起来和 memprompt 非常类似，每个问题不会直接回答答案，而是回答 understating+answer，从而让用户可以基于对 understating 的理解来判断模型反馈是否符合用户的预期，而不用直接判断 answer 的正确性

对话任务中的其他几种 memory 添加模式

ConversationSummaryMemory：与 ConversationBufferMemory 类似，不过之前的对话会被总结为一个 summary 加在 prompt 中
ConversationBufferWindowMemory：在 ConversationBufferMemory 模式基础上加个滑窗，即只加入最近几次对话的记录，避免 memory buffer 过大
ConversationSummaryBufferMemory：结合以上两种方式，将之前的对话总结为一个 summary 加在 prompt 中，同时会设置一个 prompt 最大词汇数量，超过该词汇数量的时候会抛弃更早的对话来使 prompt 的词汇数量符合要求
更高级的 memory 使用方式

Adding Memory to a Multi-Input Chain：主要用于 QA 任务，用一个语料库作为 memory，对于输入的 prompt，找到与该 prompt 类似的信息加在 prompt 中，从而能利用上语料库中的信息
Adding Memory to an Agent：对于具备 google 搜索功能的 Agent，可以将对话历史记录到 memory 中，从而能让 Agent 对某些与之前历史结合的对话理解更准确